<h1 style=\"text-align: center; font-size: 50px;\"> Interactive ORPO Fine-Tuning & Inference Hub for Open LLMs </h1>

This experiment provides an interactive and modular interface for selecting, downloading, fine-tuning, and evaluating large language models using ORPO (Optimal Reward Preferring Optimization).
The user can choose between state-of-the-art open LLMs like Mistral, LLaMA 2/3, and Gemma. 

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Model Loader
- Inference with Default Model
- Creating the Fine-Tuned Model Name (ORPO)
- Dataset Loader
- ORPO Configuration

## 📦 Imports

By using our Local GenAI workspace image, most of the necessary libraries to work with ORPO-based fine-tuning and evaluation already come pre-installed. In this notebook, we only need to import components for model loading, quantization, inference, and feedback visualization to run the complete ORPO workflow locally

In [ ]:
!pip install -r ../requirements.txt --quiet

In [ ]:
import os
import sys
import yaml
from pathlib import Path
import logging
import warnings

In [ ]:
# ===============================
# 🧠 Core Libraries
# ===============================
import torch
import multiprocessing
import mlflow
from datasets import load_dataset

# ===============================
# 🧪 Hugging Face & Transformers
# ===============================
from huggingface_hub import login
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)

# ===============================
# 🧩 Fine-tuning (ORPO + PEFT)
# ===============================
from trl import ORPOConfig, ORPOTrainer, setup_chat_format
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

# ===============================
# 🧰 Project Modules: Core Pipeline
# ===============================
# Add the core directory to the path to import utils
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from core.selection.model_selection import ModelSelector
from core.local_inference.inference import InferenceRunner
from core.target_mapper.lora_target_mapper import LoRATargetMapper
from core.data_visualizer.feedback_visualizer import UltraFeedbackVisualizer
from core.finetuning_inference.inference_runner import AcceleratedInferenceRunner
from core.merge_model.merge_lora import merge_lora_and_save
from core.quantization.quantization_config import QuantizationSelector
from core.comparer.model_comparer import ModelComparer

# ===============================
# 🚀 Deployment & Evaluation
# ===============================
from core.deploy.deploy_fine_tuning import register_llm_comparison_model

# ===============================
# ⚙️ Utility Functions
# ===============================
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    login_huggingface,
    get_project_root,
    get_config_dir,
    get_configs_dir,
    get_output_dir,
    get_models_dir,
    get_fine_tuned_models_dir,
    get_model_cache_dir,
    format_model_path,
    setup_model_environment
)

## Configurations

In [ ]:
# Suppress Python warnings
warnings.filterwarnings("ignore")

In [ ]:
CONFIG_PATH = str(get_configs_dir() / "config.yaml")
SECRETS_PATH = str(get_configs_dir() / "secrets.yaml")
MLFLOW_EXPERIMENT_NAME = "AIStudio-Fine-Tuning-Experiment"
MLFLOW_RUN_NAME = "AIStudio-Fine-Tuning-Run"
MLFLOW_MODEL_NAME = "AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_RUN_NAME="AIStudio-Fine-Tuning-Service-Run"
MODEL_SERVICE_NAME="AIStudio-Fine-Tuning-Model"
MODEL_SERVICE_EXPERIMENT_NAME="AIStudio-Fine-Tuning-Experiment"

In [ ]:
logger = logging.getLogger("fine_tuning_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
logger.info('Notebook execution started.')

### Proxy Configuration
For certain enterprise networks, a proxy configuration might be required for external service connections. If this is your case, set up the "proxy" field in your config.yaml and the following cell will configure the necessary environment variables.

In [ ]:
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

# Configure proxy using the loaded config
configure_proxy(config)

### 🔍 Model Selector

Below are the available models for fine-tuning with ORPO.  
> ⚠️ **Note:** Make sure your Hugging Face account has access permissions for the selected model (some require manual approval).

| Model ID | Hugging Face Link |
|----------|-------------------|
| `mistralai/Mistral-7B-Instruct-v0.1` | [🔗 View on Hugging Face](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) |
| `meta-llama/Llama-2-7b-chat-hf` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) |
| `meta-llama/Meta-Llama-3-8B-Instruct` | [🔗 View on Hugging Face](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) |
| `google/gemma-7b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-7b-it) |
| `google/gemma-3-1b-it` | [🔗 View on Hugging Face](https://huggingface.co/google/gemma-3-1b-it) |


In [ ]:
MODEL =  "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

### 🔐 Login to Hugging Face

To access gated models (e.g., LLaMA, Mistral, or Gemma), you must authenticate using your Hugging Face token.

Make sure your `secrets.yaml` file contains the following key:

```yaml
HUGGINGFACE_API_KEY: your_huggingface_token

In [ ]:
login_huggingface(secrets)

### Attention Optimization Config
Automatically selects the most efficient attention implementation and data type (dtype) based on the GPU’s compute capability.

In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

## Verify Assets

In [ ]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")


log_asset_status(
    asset_path=CONFIG_PATH,
    asset_name="configs.yaml",
    success_message="",
    failure_message="Please check if the configs.yaml was propely connfigured in your project on AI Studio."
)

log_asset_status(
    asset_path=SECRETS_PATH,
    asset_name="secrets.yaml",
    success_message="",
    failure_message="Please check if the secrets.yaml was propely connfigured in your project on AI Studio."
)

## Model Loader

In [ ]:
selector = ModelSelector()
selector.select_model(MODEL)

model = selector.get_model()
tokenizer = selector.get_tokenizer()


## 🤖 Inference with Default Model

The following cell runs inference using the base (non fine-tuned) model you selected earlier.

We've prepared a few prompts to test different types of reasoning and writing skills.  
You can later compare these outputs with the results generated by the fine-tuned model.

In [ ]:
# 📋 Custom prompts for evaluation
prompts = [
    "I need to write some nodejs code that publishes a message to a Telegram group.",
    "What advice would you give to a frontend developer?",
    "Propose a solution that could reduce the rate of deforestation.",
    "Write a eulogy for a public figure who inspired you."
]

# ⚙️ Run inference with the selected model
runner = InferenceRunner(selector)

for idx, prompt in enumerate(prompts, 1):
    response = runner.infer(prompt)
    print(f"\n🟢 Prompt {idx}: {prompt}\n🔽 Model Response:\n{response}\n{'-'*80}")


## 🏷️ Creating the Fine-Tuned Model Name (ORPO)

We define a clean and consistent name for the fine-tuned version of the selected base model

In [ ]:
base_model = selector.model_id
model_path = selector.format_model_path(base_model)
new_model = f"Orpo-{base_model.split('/')[-1]}-FT"
fine_tuned_name = f"Orpo-{base_model.split('/')[-1]}-FT"

fine_tuned_dir = get_fine_tuned_models_dir()
fine_tuned_dir.mkdir(parents=True, exist_ok=True)
fine_tuned_path = str(fine_tuned_dir / fine_tuned_name)

print(f"Fine-tuned model will be saved to: {fine_tuned_path}")
print(f"Directory exists: {Path(fine_tuned_path).parent.exists()}")

### ⚙️  Automatic Quantization Configuration

We use an intelligent selector to automatically choose the optimal quantization strategy for the hardware environment.

- `QuantizationSelector()` analyzes the number of available GPUs and their memory.
- If multiple GPUs with sufficient VRAM are detected, it applies 8-bit quantization for faster performance.
- Otherwise, it falls back to `4-bit QLoRA` using `nf4` and double quantization to reduce memory usage.

This adaptive configuration ensures efficient fine-tuning of large language models by balancing performance and hardware constraints.

In [ ]:
quantization = QuantizationSelector()
bnb_config = quantization.get_config()

### 🧩 PEFT Configuration (LoRA)

We define the LoRA configuration using the `LoraConfig` from PEFT (Parameter-Efficient Fine-Tuning).


In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LoRATargetMapper.get_target_modules(base_model)
)

### 🧠 Load and Prepare Base Model for Training

In this step, we load the base model and tokenizer from the local path, apply the quantization configuration (`bnb_config`), prepare it for tra

In [ ]:
model_vocab_size = AutoModelForCausalLM.from_pretrained(model_path).config.vocab_size
tokenizer_vocab_size = len(tokenizer)

if tokenizer_vocab_size != model_vocab_size:
    print(f"⚠️ Adjusting vocabulary ({tokenizer_vocab_size}) ≠ Model ({model_vocab_size})")
    tokenizer.pad_token = tokenizer.eos_token  
    tokenizer.save_pretrained(model_path)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map={"": 0},
)

In [ ]:
# Safely apply chat format only if tokenizer doesn't already have a chat_template
if tokenizer.chat_template is None:
    model, tokenizer = setup_chat_format(model, tokenizer)
else:
    print("⚠️ Tokenizer already has a chat_template. Skipping setup_chat_format to avoid overwriting.")


In [ ]:
model = prepare_model_for_kbit_training(model)


## 📚 Dataset Loader

We use the [UltraFeedback Binarized](https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized) dataset provided by Hugging Face.

This dataset contains prompts along with two model-generated responses:
- **chosen**: the response preferred by human annotators
- **rejected**: the less preferred one

For this experiment, we load a subset of the data to speed up training and evaluation.  
A fixed seed ensures reproducibility when shuffling the data.


In [ ]:
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs", "test_prefs"])

# 📊 Define sample sizes for a lightweight experiment
train_samples = 5000                         # Subset size for training
original_train_samples = 61135              # Total training examples in the original dataset
test_samples = int((2000 / original_train_samples) * train_samples)  # Proportional test size

# 🔀 Shuffle and sample subsets from both splits
train_subset = dataset[0].shuffle(seed=42).select(range(train_samples))
test_subset = dataset[1].shuffle(seed=42).select(range(test_samples))


### 📊 Dataset Visualization

To help understand how the dataset works, we use the `UltraFeedbackVisualizer`.

This tool logs examples from the dataset into **TensorBoard**, including:
- The **original prompt** given to the model
- The two possible answers: one **preferred by humans** and one that was **rejected**
- A simple comparison showing which response was rated better

Each example is displayed with clear labels and scores to help illustrate the kinds of outputs humans value more — **before we do any fine-tuning**.

> This is useful to explore what “good answers” look like, based on real human feedback.


In [ ]:
visualizer = UltraFeedbackVisualizer(train_subset, test_subset,max_samples=20)
visualizer.run()

In [ ]:
def process(row):
    """
    Specifies how to convert row into a tokenizable string in the expected model format
    """
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset[0] = train_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

dataset[1] = test_subset.map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(dataset)

## ⚙️ ORPO Configuration

We define the training configuration using the `ORPOConfig` class from TRL (Transformers Reinforcement Learning).

This configuration controls how the model will be fine-tuned using ORPO (Offline Reinforcement Preference Optimization), a technique that aligns model outputs with human preferences.

Key parameters include:
- `learning_rate`: sets how fast the model updates (8e-6 is typical for PEFT)
- `beta`: the strength of the ORPO loss term
- `optim`: uses 8-bit optimizer for memory efficiency (paged_adamw_8bit)
- `max_steps`: controls how long training will run (e.g., 1000 steps)
- `eval_strategy` and `eval_steps`: defines how and when to evaluate during training
- `output_dir`: directory to save the trained model

> This configuration is compatible with all the selected models (e.g., Mistral, LLaMA, Gemma) and optimized for QLoRA fine-tuning on consumer or research-grade GPUs.


In [ ]:
mlflow.set_tracking_uri('/phoenix/mlflow')
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

# Ensure training output directory exists
training_output_dir = get_output_dir() / "training_results"
training_output_dir.mkdir(parents=True, exist_ok=True)

orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    max_steps=1000,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to=["mlflow","tensorboard"],
    output_dir=str(training_output_dir),
)

print(f"Training output directory: {training_output_dir}")
print(f"Directory exists: {training_output_dir.exists()}")

### 🚀 ORPO Trainer

We now initialize the `ORPOTrainer`, which orchestrates the fine-tuning process using the Offline Reinforcement Preference Optimization (ORPO) strategy.

It takes as input:
- The **base model**, already prepared with QLoRA and chat formatting
- The **ORPO configuration** (`orpo_args`) containing all training hyperparameters
- The **training and evaluation datasets**
- The **LoRA configuration** (`peft_config`) for parameter-efficient fine-tuning
- The **tokenizer**, passed as a `processing_class`, to apply proper formatting and padding

Once initialized, the trainer will be ready to start training with `trainer.train()`.


In [ ]:
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset[0],
    eval_dataset=dataset[1],
    peft_config=peft_config,
    processing_class=tokenizer  
)

In [ ]:
trainer.train()

# Copy the final model to our desired fine_tuned_path location
import shutil
if Path(orpo_args.output_dir).exists():
    # Remove existing fine_tuned_path if it exists
    if Path(fine_tuned_path).exists():
        shutil.rmtree(fine_tuned_path)
    
    # Copy the trained model to our desired location
    shutil.copytree(orpo_args.output_dir, fine_tuned_path)
    print(f"Model copied to: {fine_tuned_path}")

In [ ]:
# Find the LoRA adapters in the training output directory
training_output_dir = Path(orpo_args.output_dir)

# Look for adapter_config.json in checkpoint subdirectories
adapter_configs = list(training_output_dir.rglob("adapter_config.json"))
if adapter_configs:
    # Use the directory containing adapter_config.json (typically checkpoint-X)
    lora_adapter_path = str(adapter_configs[0].parent)
    print(f"Found LoRA adapters at: {lora_adapter_path}")
    
    # Merge LoRA adapters with base model
    merge_lora_and_save(
        base_model_id=MODEL,
        finetuned_lora_path=lora_adapter_path
    )
else:
    print("❌ No LoRA adapters found in training output directory!")
    print("This might indicate an issue with the training process.")

In [ ]:
# Load the merged fine-tuned model for inference
final_model_path = str(get_fine_tuned_models_dir() / fine_tuned_name)

print(f"Loading fine-tuned model from: {final_model_path}")

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(final_model_path)
model = AutoModelForCausalLM.from_pretrained(final_model_path, torch_dtype=torch.float16).cuda().eval()

# Test the fine-tuned model with a sample prompt
prompt = "Propose a solution that could reduce the rate of deforestation"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=500)

print("\nFine-tuned Model Response:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 🔍 Model Evaluation and Comparison

After completing the ORPO fine-tuning process, we can evaluate the performance improvements by comparing responses from the base model and our fine-tuned model.

This comparison helps us understand:
- **Quality Improvements**: How the fine-tuned model generates more helpful and aligned responses
- **Training Effectiveness**: Whether the ORPO training successfully improved the model's preference alignment
- **Response Consistency**: How well the model maintains coherent and relevant outputs

The comparison uses the same test prompts to ensure fair evaluation between the base and fine-tuned models.

In [ ]:
# Compare base model vs fine-tuned model using ModelComparer
final_model_path = str(get_fine_tuned_models_dir() / fine_tuned_name)

print("🔍 MODEL COMPARISON RESULTS")
print("=" * 50)

# Initialize the ModelComparer
comparer = ModelComparer()

# Load both models
base_model_selector = ModelSelector()
print(f"📊 Base model: {base_model_selector.model_id}")
print(f"📊 Fine-tuned model: {final_model_path}")

# Define test prompts for comparison
test_prompts = [
    "Explain the importance of sustainable agriculture.",
    "Write a Python function to check for palindromes.",
    "Describe the benefits of renewable energy sources.",
    "What are the key principles of machine learning?"
]

# Run comparison using ModelComparer
print("\n🚀 Running model comparison...")
comparison_results = comparer.compare_models(
    base_model_path=base_model_selector.format_model_path(base_model_selector.model_id),
    finetuned_model_path=final_model_path,
    prompts=test_prompts,
    max_new_tokens=150
)

# Display results
print("\n📝 COMPARISON RESULTS:")
print("=" * 50)

for i, result in enumerate(comparison_results, 1):
    print(f"\n--- Test Prompt {i} ---")
    print(f"Prompt: {result['prompt']}")
    print(f"\n🔸 Base Model Response:")
    print(result['base_response'])
    print(f"\n🔹 Fine-tuned Model Response:")
    print(result['finetuned_response'])
    print("-" * 30)

print("\n✅ Model comparison completed successfully!")

In [ ]:
logger.info('Notebook execution completed.')

Built with ❤️ using Z by HP AI Studio.